# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [2]:
from pyspark.sql import (SparkSession,functions as f,types as t)
spark = SparkSession.builder.appName('excerise').getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [3]:
df_walmart = spark.read.csv('walmart_stock.csv',inferSchema = True,
                            header = True)

#### What are the column names?

In [4]:
df_walmart.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [5]:
df_walmart.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [6]:
df_walmart.head(5)

[Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

#### Use describe() to learn about the DataFrame.

In [7]:
df_stats = df_walmart.describe()

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [8]:
df_stats.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [9]:
df_stats = df_stats.withColumn('Open',
                    f.format_number((df_stats.Open.cast(t.DoubleType())),
                                     2))
df_stats = df_stats.withColumn('Close',
                    f.format_number((df_stats.Close.cast(t.DoubleType())),
                                     2))
df_stats = df_stats.withColumn('High',
                    f.format_number((df_stats.High.cast(t.DoubleType())),
                                     2))
df_stats = df_stats.withColumn('Low',
                    f.format_number((df_stats.Low.cast(t.DoubleType())),
                                     2))
df_stats = df_stats.withColumn('Volume',
                    f.format_number((df_stats.Volume.cast(t.DoubleType())),
                                     2))
df_stats = df_stats.withColumn('Adj Close',
                    f.format_number((f.col('Adj Close').cast(t.DoubleType())),
                                     2))

In [16]:
df_stats.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [14]:
df_hv_ratio = df_stats
df_hv_ratio = df_hv_ratio.withColumn('HV_Ratio',f.col('High')/f.col('Volume'))
df_hv_ratio.select('HV_Ratio').show()

+--------+
|HV_Ratio|
+--------+
|    null|
|    null|
|    null|
|    null|
|    null|
+--------+



#### What day had the Peak High in Price?

In [29]:
df_hv_ratio = df_hv_ratio.withColumn('High',f.col('High').cast(t.DoubleType()))
peak_day = df_hv_ratio.groupby('Date').sum('High').orderBy(f.col('sum(High)').desc()).collect()[0][0]
print(peak_day)

2016-12-30


#### What is the mean of the Close column?

In [34]:
close_mean = df_hv_ratio.agg({'Close':'avg'})
close_mean.show()

+----------+
|avg(Close)|
+----------+
|     56.51|
+----------+



#### What is the max and min of the Volume column?

In [36]:
df_hv_ratio.agg(f.max('Volume'),f.min('Volume')).show()

+-------------+-----------+
|  max(Volume)|min(Volume)|
+-------------+-----------+
|80,898,100.00|   1,258.00|
+-------------+-----------+



+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [39]:
df_hv_ratio = df_hv_ratio.withColumn('Close',f.col('Close').cast(t.DoubleType()))
df_hv_ratio.filter(f.col('Close')<60).show()

+-------+----------+-----+-----+-----+-----+------------+---------+--------+
|summary|      Date| Open| High|  Low|Close|      Volume|Adj Close|HV_Ratio|
+-------+----------+-----+-----+-----+-----+------------+---------+--------+
| stddev|      null| 6.77| 6.77| 6.74| 6.76|4,519,780.84|     6.72|    null|
|    min|2012-01-03|56.39|57.06|56.30|56.42|2,094,900.00|    50.36|    null|
+-------+----------+-----+-----+-----+-----+------------+---------+--------+



#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [42]:
df_hv_ratio = df_hv_ratio.withColumn('Month',f.month('Date'))
df_hv_ratio.groupBy('Month').agg({'Close':'avg'}).\
            orderBy(f.col('Month')).show()

+-----+----------+
|Month|avg(Close)|
+-----+----------+
| null|    39.575|
|    1|     56.42|
|   12|     90.47|
+-----+----------+



# Great Job!